In [1]:
import numpy as np
from skimage.util.shape import view_as_windows
from numpy.lib.stride_tricks import as_strided

##### Input Layer

In [2]:
input_data_list = [[-1, 1, 1, 1,-1],
                   [-1, 1,-1, 1,-1],
                   [-1, 1, 1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1, 1,-1,-1],
                   [-1, 1,-1,-1,-1]]

# Convert list to input data matrix
input_data_matrix = np.asarray(a=input_data_list, dtype=np.float32)

print(input_data_matrix.shape)

(7, 5)


##### Convolution Layer(convolution operation)

In [3]:
box_filter_list = [[1, 1, 1],
                   [1,-1, 1],
                   [1, 1, 1]]

vertical_line_filter_list = [[-1, 1,-1],
                      [-1, 1,-1],
                      [-1, 1,-1]]

diagonal_line_filter_list = [[-1,-1,-1],
                             [-1, 1,-1],
                             [ 1,-1,-1]]

In [4]:
# Convert all filter lists into matrices
box_filter_matrix = np.asarray(box_filter_list, dtype=np.float32)
vertical_line_filter_matrix = np.asarray(vertical_line_filter_list, dtype=np.float32)
diagonal_line_filter_matrix = np.asarray(diagonal_line_filter_list, dtype=np.float32)

print(f"box_filter_matrix's shape : {box_filter_matrix.shape}")
print(f"vertical_line_filter_matrix's shape : {vertical_line_filter_matrix.shape}")
print(f"diagonal_line_filter_matrix's shape : {diagonal_line_filter_matrix.shape}")

box_filter_matrix's shape : (3, 3)
vertical_line_filter_matrix's shape : (3, 3)
diagonal_line_filter_matrix's shape : (3, 3)


In [10]:
# Extract each window from input matrix by stride operation
def strided4D_v2(input_matrix, kernel_matrix, stride):
    return view_as_windows(input_matrix, kernel_matrix.shape, step=stride)

In [11]:
# Calculate shape of the feature map (output matrix from convolution layer)   # (5, 3, 3, 3) : feature map's shape is (5, 3) ==> (5, 3, 3, 3)
featureMap_row = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[0]  # 5
featureMap_col = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[1]  # 3

print(f"featureMap_row : {featureMap_row},   featureMap_col : {featureMap_col}")

featureMap_row : 5,   featureMap_col : 3


In [ ]:
# Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernel_matrix):
    # Create blank featuremap matrix for stride 1
    featureMap_output = np.zeros((featureMap_row, featureMap_col))

In [ ]:
# Function to Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernal_matrix):
    # Create blank featureMap matrix for stride 1
    featureMap_Output = np.zeros((featureMap_row, featureMap_col))
 
    for row in range(featureMap_row):
        for col in range(featureMap_col):
            window = strided4D_v2(input_matrix, kernal_matrix, 1)[row][col]
            featureMap_Output[row, col] = np.sum(np.multiply(kernal_matrix, window))
 
            # # To Format floats in a numpy array
            np.set_printoptions(precision=2)
            # Taking average with divided by 9 (total number of element in filter matrix)
            total_number_of_element_in_filter_matrix = kernal_matrix.shape[0] * kernal_matrix.shape[0]
    return (featureMap_Output / total_number_of_element_in_filter_matrix)

In [ ]:
# *******************************************************************************
# Convolution layer (convolution operation)
# *******************************************************************************
 
box_filter_list = [
[1, 1,1],
[1,-1,1],
[1, 1,1]
 
]
 
vertical_line_Filter_list = [
[-1, 1, -1],
[-1, 1, -1],
[-1, 1, -1]
 
]
 
Diagonal_line_Filter_list = [
[-1, -1, 1],
[-1, 1, -1],
[1, -1, -1]
 
]
 
# Convert all filter list into matrix
box_filter_matrix = np.asarray(box_filter_list, dtype=np.float32)
vertical_line_Filter_matrix = np.asarray(vertical_line_Filter_list, dtype=np.float32)
Diagonal_line_Filter_matrix = np.asarray(Diagonal_line_Filter_list, dtype=np.float32)
 
# Extract each window from input matrix  by stride operation
def strided4D_v2(input_image_matrix,kernal_matrix,stride):
    return view_as_windows(input_image_matrix, kernal_matrix.shape, step=stride)
 
# Calculate shape of the feature map (output matrix from convolution layer)
featureMap_row = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[0]
featureMap_col = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[1]
 
# Function to Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernal_matrix):
    # Create blank featureMap matrix for stride 1
    featureMap_Output = np.zeros((featureMap_row, featureMap_col))
 
    for row in range(featureMap_row):
        for col in range(featureMap_col):
            window = strided4D_v2(input_matrix, kernal_matrix, 1)[row][col]
            featureMap_Output[row, col] = np.sum(np.multiply(kernal_matrix, window))
 
            # # To Format floats in a numpy array
            np.set_printoptions(precision=2)
            # Taking average with divided by 9 (total number of element in filter matrix)
            total_number_of_element_in_filter_matrix = kernal_matrix.shape[0] * kernal_matrix.shape[0]
    return (featureMap_Output / total_number_of_element_in_filter_matrix)
 
# ------------------------------------------------------------------------
# Box Filter operation
# ------------------------------------------------------------------------
# Create blank featureMap matrix for stride 1
featureMap_Box = conv2d(input_data_matrix, box_filter_matrix)
print(featureMap_Box)
 
# ------------------------------------------------------------------------
# Vertical line Filter operation
# ------------------------------------------------------------------------
featureMap_Vertical = conv2d(input_data_matrix, vertical_line_Filter_matrix)
# print(featureMap_Vertical)
 
# ------------------------------------------------------------------------
# Diagonal line Filter operation
# ------------------------------------------------------------------------
# Output after applying Diagonal line Filter with stride 1
featureMap_Diagonal = conv2d(input_data_matrix, Diagonal_line_Filter_matrix)
# print(featureMap_Diagonal)